## Objetivo

Expandir o DataFrame com os avisos antigos contendo *teor_texto*, *assuntos*, *classe_processual* e *orgao_julgador*

In [121]:
import pandas as pd
import os

path = 'data/raw'

df_avisos = pd.read_csv(os.path.join(path, 'df_avisos.csv'), dtype=str)
df_processo_assuntos = pd.read_csv(os.path.join(path, 'df_processo_assuntos.csv'), dtype=str)
df_processo_dados = pd.read_csv(os.path.join(path, 'df_processo_dados.csv'), dtype=str)
df_tramitacoes_pje_janeiro_maio = pd.read_excel(os.path.join(path, 'tramitacoes_pje_janeiro_maio_2025.xls'), dtype=str)


In [122]:
# Colunas de cada DF
print(f'Colunas df_avisos: {df_avisos.columns}')
print(f'Colunas df_processo_assuntos: {df_processo_assuntos.columns}')
print(f'Colunas df_processo_dados: {df_processo_dados.columns}')
print(f'Colunas df_tramitacoes_pje_janeiro_maio: {df_tramitacoes_pje_janeiro_maio.columns}')

Colunas df_avisos: Index(['Unnamed: 0', 'id_aviso', 'id_processo', 'tipo_prazo',
       'data_referencia', 'id_movimento', 'id_documento', 'tipo_documento',
       'data_hora_documento', 'movimento_documento', 'descricao_documento',
       'teor_texto'],
      dtype='object')
Colunas df_processo_assuntos: Index(['Unnamed: 0', 'id_processo', 'assunto', 'principal'], dtype='object')
Colunas df_processo_dados: Index(['Unnamed: 0', 'id_processo', 'cod_competencia', 'cod_classe_processual',
       'cod_localidade', 'nivel_sigilo', 'data_ajuizamento', 'valor_causa',
       'orgao_julgador', 'status_processo', 'polo_ativo', 'polo_passivo'],
      dtype='object')
Colunas df_tramitacoes_pje_janeiro_maio: Index(['idavisopje', 'numeroprocessojudicial', 'assuntodirecta',
       'datarecebimentoaviso', 'setordestino', 'dataenvioDestino'],
      dtype='object')


O que nos interessa é:

* Em df_avisos: *id_aviso*, *id_processo* e *teor_texto*
* Em df_processo_assuntos: *id_processo* e *assunto* (Juntar no formato [assunto1;assunto2]) 
* Em df_processo_dados: *id_processo*, *cod_classe_processual* e *orgao_julgador*
* Em df_tramitacoes_pje_janeiro_maio: *idavisopje*, *datarecebimentoaviso* e **setordestino**

In [123]:
df_avisos_filtered = df_avisos[['id_aviso', 'id_processo', 'teor_texto']]
df_processo_assuntos_filtered = df_processo_assuntos[['id_processo', 'assunto']]
df_processo_dados_filtered = df_processo_dados[['id_processo', 'cod_classe_processual', 'orgao_julgador']]
df_tramitacoes_pje_janeiro_maio = df_tramitacoes_pje_janeiro_maio[['idavisopje', 'datarecebimentoaviso', 'setordestino']]

In [124]:
# Transformando o df_processo_assuntos_filtered para o formato [assunto1;assunto2]

df_processo_assuntos_filtered = (
    df_processo_assuntos_filtered.groupby('id_processo')['assunto']
        .apply(lambda x: ';'.join(map(str, x)))  # Converte para string antes de juntar
        .reset_index()
)
df_processo_assuntos_filtered.rename(columns={'assunto':'assuntos'}, inplace=True)
df_tramitacoes_pje_janeiro_maio.rename(
    columns={
        'idavisopje':'id_aviso',
        'datarecebimentoaviso':'data_aviso',
        'setordestino':'setor_destino'
    }, 
    inplace=True
)

In [126]:
# Junta avisos + assuntos
df_merge = pd.merge(
    df_avisos_filtered,
    df_processo_assuntos_filtered,
    on='id_processo',
    how='left'
)

df_merge

,id_aviso,id_processo,teor_texto,assuntos
0,21536453,08456673920248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299
1,22950420,08298182720248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10433
2,22948927,08612110420238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,6062
3,21983711,08239510520148205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10536;5952
4,21693281,08533260220248205001,Poder Judiciário do Estado do Rio Grande do No...,10236
...,...,...,...,...
34850,21724374,08606836720238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,14201
34851,22132800,08072055720178205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,5952;6013
34852,21662712,08873817620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299
34853,22333288,08006030620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10236


In [127]:
# Junta merge + dados do processo

df_merge = pd.merge(
    df_merge,
    df_processo_dados_filtered,
    on='id_processo',
    how='left'
)

df_merge

,id_aviso,id_processo,teor_texto,assuntos,cod_classe_processual,orgao_julgador
0,21536453,08456673920248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,12078,2º Juizado da Fazenda Pública da Comarca de Natal
1,22950420,08298182720248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10433,12078,2º Juizado da Fazenda Pública da Comarca de Natal
2,22948927,08612110420238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,6062,12078,4º Juizado da Fazenda Pública da Comarca de Natal
3,21983711,08239510520148205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10536;5952,1116,2ª Vara de Execução Fiscal e Tributária de Natal
4,21693281,08533260220248205001,Poder Judiciário do Estado do Rio Grande do No...,10236,14695,5º Juizado da Fazenda Pública da Comarca de Natal
...,...,...,...,...,...,...
34850,21724374,08606836720238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,14201,12078,6º Juizado da Fazenda Pública da Comarca de Natal
34851,22132800,08072055720178205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,5952;6013,156,5ª Vara de Execução Fiscal e Tributária de Natal
34852,21662712,08873817620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,14695,6º Juizado da Fazenda Pública da Comarca de Natal
34853,22333288,08006030620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10236,12078,3º Juizado da Fazenda Pública da Comarca de Natal


In [135]:
df_final = pd.merge(
    df_merge,
    df_tramitacoes_pje_janeiro_maio,
    on='id_aviso',
    how='left'
)

# Verificando se existem campos nulos no setor_destino
print(df_final.isnull().sum())

id_aviso                 0
id_processo              0
teor_texto               0
assuntos                 7
cod_classe_processual    4
orgao_julgador           4
data_aviso               0
setor_destino            0
dtype: int64


In [136]:
df_final.to_csv(os.path.join(path, 'novo_dataframe_com_metadados.csv'), index=False)

df_final

,id_aviso,id_processo,teor_texto,assuntos,cod_classe_processual,orgao_julgador,data_aviso,setor_destino
0,21536453,08456673920248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,12078,2º Juizado da Fazenda Pública da Comarca de Natal,2025-01-23 14:54:46.270,Procuradoria Administrativa
1,22950420,08298182720248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10433,12078,2º Juizado da Fazenda Pública da Comarca de Natal,2025-04-30 03:00:17.540,DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE
2,22948927,08612110420238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,6062,12078,4º Juizado da Fazenda Pública da Comarca de Natal,2025-04-30 03:00:17.540,DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE
3,21983711,08239510520148205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10536;5952,1116,2ª Vara de Execução Fiscal e Tributária de Natal,2025-02-21 03:00:20.063,Procuradoria Fiscal
4,21693281,08533260220248205001,Poder Judiciário do Estado do Rio Grande do No...,10236,14695,5º Juizado da Fazenda Pública da Comarca de Natal,2025-02-04 03:00:13.920,Procuradoria Administrativa
...,...,...,...,...,...,...,...,...
34898,21724374,08606836720238205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,14201,12078,6º Juizado da Fazenda Pública da Comarca de Natal,2025-02-06 03:00:18.950,REQUISITÓRIO DE PAGAMENTO - CONTABILIDADE
34899,22132800,08072055720178205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,5952;6013,156,5ª Vara de Execução Fiscal e Tributária de Natal,2025-03-07 03:00:17.030,Procuradoria Fiscal
34900,21662712,08873817620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,14695,6º Juizado da Fazenda Pública da Comarca de Natal,2025-02-03 03:00:18.300,Procuradoria Administrativa
34901,22333288,08006030620248205001,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10236,12078,3º Juizado da Fazenda Pública da Comarca de Natal,2025-03-19 03:00:17.377,DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE
